In [43]:
import sys, os
import rasterio

import geopandas as gpd
import pandas as pd

sys.path.insert(0, "c:\\WBG\\Work\\Code\\GOSTrocks\\src")
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.rasterMisc as rMisc

sys.path.insert(0, "../src")

import global_zonal
import h3_helper

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import pickle

pickle_file = "h1_dictionary_of_h6_geodata_frames.pickle"
with open(pickle_file, 'wb') as f:
    pickle.dump(h3_1_list, f)

In [54]:
AWS_S3_BUCKET = 'wbg-geography01'
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

h3_level = 6
h3_1_list = h3_helper.generate_lvl0_lists(h3_level, return_gdf=True, buffer0=False, read_pickle=False)

pop_layer = r"C:\WBG\Work\data\POP\ppp_2020_1km_Aggregated.tif"
in_pop = rasterio.open(pop_layer)



TypeError: generate_lvl0_lists() got an unexpected keyword argument 'read_pickel'

In [55]:
# Select layer to downlaod
flood_type = ["PLUVIAL","FLUVIAL","COASTAL"]
defence = ["DEFENDED"]
return_period = ['1in100']
climate_model = ["PERCENTILE50"]
year = ["2020"]

# all_vrts is a pandas dataframe with all the vrt paths to the global datasets, with columns defining
# the various models' defining attributes
all_vrts = dMisc.get_fathom_vrts(True)
sel_images = all_vrts.loc[(all_vrts['FLOOD_TYPE'].isin(flood_type)) & (all_vrts['DEFENCE'].isin(defence)) & 
             (all_vrts['RETURN'].isin(return_period))  & (all_vrts['CLIMATE_MODEL'].isin(climate_model))]
fathom_vrt_path = sel_images['PATH'].iloc[0]
in_fathom = rasterio.open(fathom_vrt_path)

In [56]:
sel_images.head()

,RETURN,FLOOD_TYPE,DEFENCE,DEPTH,YEAR,CLIMATE_MODEL,PATH
65,1in100,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
91,1in100,FLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
117,1in100,PLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...


In [57]:
for fathom_index, fathom_row in sel_images.iterrows():
    out_file = "_".join([fathom_row['FLOOD_TYPE'], fathom_row['RETURN'], fathom_row['CLIMATE_MODEL'], fathom_row['YEAR']])
    print(out_file)


COASTAL_1in100_PERCENTILE50_2020
FLUVIAL_1in100_PERCENTILE50_2020
PLUVIAL_1in100_PERCENTILE50_2020


In [41]:
sample_id = list(h3_1_list.keys())[0]
out_file = f"C:/WBG/Work/data/zonal_{sample_id}.csv"
reclass_dict = { 0: [-9999, 0], 
                1: [0, 10], 
                2: [10.1, 50], 
                3: [50, 100000.0],}

xx = global_zonal.zonal_stats_categorical(h3_1_list[sample_id], "shape_id", in_pop, in_fathom, out_file, reclass_dict=reclass_dict, 
                                          buffer0=True, minVal=0, maxVal=10000000)

In [48]:
global_zonal.zonal_stats_numerical()

{'C:/WBG/Work/data/zonal_8140fffffffffff.csv':                        0_SUM  0_MIN        0_MAX     0_MEAN         1_SUM  \
 id                                                                          
 8640d50f7ffffff     0.000000    0.0     0.000000   0.000000   2067.635986   
 8640f63a7ffffff     0.000000    0.0     0.000000   0.000000   5794.220215   
 8640ce0e7ffffff     0.000000    0.0     0.000000   0.000000     54.516670   
 8640d080fffffff     0.000000    0.0     0.000000   0.000000  18001.263672   
 8640e26c7ffffff   234.824295    0.0   234.824295   4.996262  22601.615234   
 ...                      ...    ...          ...        ...           ...   
 8640ca7afffffff     0.000000    0.0     0.000000   0.000000     77.144470   
 8640d9227ffffff     0.000000    0.0     0.000000   0.000000    201.811218   
 8640e3c2fffffff  1834.572021    0.0  1834.572021  39.882000  76966.687500   
 8640ced27ffffff     0.000000    0.0     0.000000   0.000000    383.207001   
 8640f032fffffff  

# DEBUGGING

In [45]:
# clip and extract the input datasets
out_folder = "C:/WBG/Work/S2S/data/FATHOM_TESTING"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

out_fathom = os.path.join(out_folder, f"fathom_{sample_id}.tif")
out_pop = os.path.join(out_folder, f"pop_{sample_id}.tif")
out_h3 = os.path.join(out_folder, f"h3_{sample_id}.geojson")
if not os.path.exists(out_h3):
    h3_1_list[sample_id].to_file(out_h3, driver='GeoJSON')

if not os.path.exists(out_pop):
    rMisc.clipRaster(in_pop, h3_1_list[sample_id], out_pop)

if not os.path.exists(out_fathom):
    temp_p = rasterio.open(out_pop)
    rMisc.standardizeInputRasters(in_fathom, temp_p, out_fathom)




In [42]:
rMisc.clipRaster()

Object `rMisc.clipRaster` not found.


# clip and extract the input datasets
out_folder = "C:/